In [1]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess
import json


In [ ]:
        
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
device=torch.device('cpu')
metrics_type=['nn', 'structural','molecular', 'auc_roc']
#leave it blank to compute all the structural structures 
structural_statistic=['nspdk','WL'] 


In [ ]:
from rdkit import RDLogger 

def get_data(name, path='data/smiles/',return_smiles=False):
    splits={}
    RDLogger.EnableLog('rdApp.*')
    RDLogger.DisableLog('rdApp.*')  
    split_names=[ 'train1_pos_smiles','train1_neg_smiles','test_smiles','test_targets' ]
    for i,split in enumerate(split_names):
        exact_path=path+'{}/{}.txt'.format(name, split)
        #from data.smiles.carcinogens import test_smiles
        current_list = []
        with open(exact_path) as my_file:
         for line in my_file:
            current_list.append(line.strip())
        splits[split]=current_list     
     


    return splits
    

In [ ]:
splits=get_data('ames', return_smiles=True)
train1_pos_smiles,train1_neg_smiles, test_smiles, test_targets=splits['train1_pos_smiles'], splits['train1_neg_smiles'],splits['test_smiles'],splits['test_targets']
test_pos=[x for x, y in zip(splits['test_smiles'],splits['test_targets']) if y=='1']
test_neg=[x for x, y in zip(splits['test_smiles'],splits['test_targets']) if y=='0']

In [ ]:
from evaluation.moses.metrics.utils  import logP, QED, SA
from rdkit import Chem
import pandas as pd
data=train1_pos_smiles+test_pos
mols =[Chem.MolFromSmiles(smiles)  for smiles in data]
log_p =[logP(mol) for mol in mols]
qeds= [QED(mol) for mol in mols]
sas= [SA(mol) for mol in mols]
dataset_name='ames'
d1={'smiles':data,'logP':log_p, 'qed':qeds,'SAS':sas}
trial=pd.DataFrame(data=d1)
trial.to_csv('{}_train1_pos.csv'.format(dataset_name), index=True)

In [ ]:
data2=train1_neg_smiles+test_neg
mols =[Chem.MolFromSmiles(smiles)  for smiles in data]
log_p =[logP(mol) for mol in mols]
qeds= [QED(mol) for mol in mols]
sas= [SA(mol) for mol in mols]
d2={'smiles':data2,'logP':log_p, 'qed':qeds,'SAS':sas}
trial=pd.DataFrame(data=d2)
trial.to_csv('{}_train1_neg.csv'.format(dataset_name), index=True)

In [ ]:
len(test_pos)

798

In [ ]:
len(data)

2175

In [ ]:
len(train1_pos_smiles)

1377

In [ ]:
len(train1_pos_smiles)

1377

In [ ]:
len(test_pos)

798

In [ ]:
valid_idx_pos=list(np.arange (len(train1_pos_smiles), len(train1_pos_smiles)+ len(test_pos)))
valid_idx_pos_=[str(i) for i in valid_idx_pos]
valid_idx_neg=list(np.arange (len(train1_neg_smiles), len(train1_neg_smiles)+ len(test_neg)))
valid_idx_neg_=[str(i) for i in valid_idx_neg]

In [ ]:
dict={"valid_idxs":valid_idx_pos_}
dict


{'valid_idxs': ['1377',
  '1378',
  '1379',
  '1380',
  '1381',
  '1382',
  '1383',
  '1384',
  '1385',
  '1386',
  '1387',
  '1388',
  '1389',
  '1390',
  '1391',
  '1392',
  '1393',
  '1394',
  '1395',
  '1396',
  '1397',
  '1398',
  '1399',
  '1400',
  '1401',
  '1402',
  '1403',
  '1404',
  '1405',
  '1406',
  '1407',
  '1408',
  '1409',
  '1410',
  '1411',
  '1412',
  '1413',
  '1414',
  '1415',
  '1416',
  '1417',
  '1418',
  '1419',
  '1420',
  '1421',
  '1422',
  '1423',
  '1424',
  '1425',
  '1426',
  '1427',
  '1428',
  '1429',
  '1430',
  '1431',
  '1432',
  '1433',
  '1434',
  '1435',
  '1436',
  '1437',
  '1438',
  '1439',
  '1440',
  '1441',
  '1442',
  '1443',
  '1444',
  '1445',
  '1446',
  '1447',
  '1448',
  '1449',
  '1450',
  '1451',
  '1452',
  '1453',
  '1454',
  '1455',
  '1456',
  '1457',
  '1458',
  '1459',
  '1460',
  '1461',
  '1462',
  '1463',
  '1464',
  '1465',
  '1466',
  '1467',
  '1468',
  '1469',
  '1470',
  '1471',
  '1472',
  '1473',
  '1474',
  '147

In [ ]:
with open("valid_idx_ames_train1_pos.json", "w") as outfile:
    json.dump(dict, outfile)